# Inspect decaying trajectories

In [1]:
import numpy as np
import os

In [2]:
%matplotlib qt
import matplotlib.pyplot as plt

In [5]:
%load_ext autoreload
%autoreload 2
import utils as utils
import plot_utils as plot

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Select folder where decay trajectories (.pkl files) are located and location of rest file (containing equilibrium position)

In [6]:
diamond_dir = "../../../soft-robot-control/examples/diamond"
decayData_dir = os.path.join(diamond_dir, "dataCollection/pre-tensioned")
rest_file = os.path.join(diamond_dir, 'rest.pkl')

Load all trajectories

In [9]:
output_node = 1354
t_in = 0
t_out = 5

oData = utils.import_pos_data(decayData_dir, rest_file, output_node, t_in, t_out, return_velocity=True)
Data = {}
Data['oData'] = oData

Plot decay data in (x, y, z)

In [10]:
# outdofs = [0, 1, 2]
plt.close('all')
# plot trajectories in 3D [x, y, z] space
plot.traj_3D(Data,
             xyz_idx=[('oData', 0), ('oData', 1), ('oData', 2)],
             xyz_names=[r'$x$', r'$y$', r'$z$'])
# plot evolution of x, y and z in time, separately in 3 subplots
highlight_traj = []
plot.traj_xyz(Data,
             xyz_idx=[('oData', 0), ('oData', 1), ('oData', 2)],
             xyz_names=[r'$x$', r'$y$', r'$z$'],
             highlight_idx=highlight_traj)
# plot velocities
plot.traj_xyz(Data,
             xyz_idx=[('oData', 3), ('oData', 4), ('oData', 5)],
             xyz_names=[r'$x$', r'$y$', r'$z$'],
             highlight_idx=highlight_traj)

ValueError: x and y must have same first dimension, but have shapes (500,) and (1000,)

In [ ]:
# print(Data['oData'][0][0])
print(Data['oData'][0][1][:, -1])

In [ ]:
for i in range(len(Data['oData'])):
    np.savetxt(f"data_roshan/decayTraj_pre-tensioned_{i:02d}.csv", Data['oData'][i][1])
